<a href="https://colab.research.google.com/github/penelopetir/BIOL74-Final-Project/blob/main/Final_Project_Penelope_and_Andy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before starting, we'll install neuron in our current runtime as usual.

In [1]:
%pip install neuron # only need to run this cell once to install
                    # neuron in the local jupyter environment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 52.4 MB/s eta 0:00:00


Run the code block below just once to get all the files from the repository into our colab session and compile the MOD mechanism files we'll be using

In [3]:
repo_name = 'BIOL74-Final-Project'
if 'google.colab' in str(get_ipython()):
    import os
    if not os.path.exists(repo_name):
        !git clone https://github.com/penelopetir/{repo_name}.git # downloads repository into our Google colab session's file system

    os.chdir(repo_name) # Changing working directory to downloaded repository

# Compile mechanisms
!nrnivmodl mechanisms

/content/BIOL74-Final-Project
Mod files: "mechanisms/mechanisms/CadepK.mod" "mechanisms/mechanisms/CaDynamics_E2.mod" "mechanisms/mechanisms/Ca_HVA.mod" "mechanisms/mechanisms/Ca_LVAst.mod" "mechanisms/mechanisms/Ca.mod" "mechanisms/mechanisms/Ih.mod" "mechanisms/mechanisms/Im.mod" "mechanisms/mechanisms/K_Pst.mod" "mechanisms/mechanisms/K_Tst.mod" "mechanisms/mechanisms/Nap_Et2.mod" "mechanisms/mechanisms/NaTa_t.mod" "mechanisms/mechanisms/ProbAMPANMDA.mod" "mechanisms/mechanisms/ProbGABAA.mod" "mechanisms/mechanisms/SK_E2.mod" "mechanisms/mechanisms/SKv3_1.mod"

 -> Compiling mod_func.cpp
 => LINKING shared library ./libnrnmech.so
Successfully created x86_64/special


## Introduction
For this exercise, we'll be adapting a model of a L5PC to model granule neurons traced from wild type and PTEN KO groups


Start by importing the packages we'll be using.

In [4]:
from neuron import h # all NEURON hoc functions are accessed through h
from neuron import gui # if you're running this notebook in a local environment (instead of on google colab), launches the GUI
h.load_file('stdrun.hoc') # loads neuron standard hoc library, not always necessary
import numpy as np
import matplotlib as mpl
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
import matplotlib.pyplot as plt
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go

if 'google.colab' in str(get_ipython()): # For making interactive plots work on google colab
    from google.colab import output
    output.enable_custom_widget_manager()
else:
    plotly.offline.init_notebook_mode() # for printing notebooks offline

#Importing Morphologies

Files from the github of traced neurons from NeuroLuicda 360 are imported and created with proper morphology. Biophysics are modeled after the Aradi et al 1999 paper.

In [8]:
from helper_functions import createGranuleCell

genotype = "WT" #change this string to "WT" or "KO" to determine which neuron to run the following code on

if genotype == "WT":
  WT_file = "morphologies/WT.asc" # morphology file
  GC = createGranuleCell(WT_file)
elif genotype == "KO":
  KO_file = "morphologies/PTEN_KO.asc" # morphology file
  GC = createGranuleCell(KO_file)
else:
  print("No neuron found")

# Create recording vectors for time and somatic voltage
t_vec = h.Vector().record(h._ref_t)
v_soma = h.Vector().record(GC.soma[0](0.5)._ref_v)

somastack_process 5
center
213.285	-110.197	-70	
Principle values
129.432	11.5282	3.74393	
Principle axes
 0.0589   -0.931   -0.36   
 -0.268   -0.362   0.893   
 -0.962   0.0437   -0.271  
stk_triang_area = 266.091


In [12]:
#probing neurons for their parameters
print(f"{genotype} Soma Ra: {GC.soma[0].Ra} MOhm")
print(f"{genotype} Soma L: {GC.soma[0].L:.2f} um")
print(f"{genotype} Soma Diameter: {GC.soma[0].diam:.2f} um")
print(f"{genotype} Soma Cm: {GC.soma[0].cm} pF")

WT Soma Ra: 210.0 MOhm
WT Soma L: 8.22 um
WT Soma Diameter: 9.29 um
WT Soma Cm: 1.0 pF


#Measuring Rin and Cm
In this next section, we will be calibrating our model in an attempt to match the Williams et al 2015 paper results. The objective is to get our values of Rm and Cm to be as such:

WT:

KO:

The approach we will be taking is to apply a current stimulus and then measure the change in voltage of the soma membrane. With this, we can use Ohm's law to calculate the resistance by dividing the change in voltage by the current.

From here, we can take this value and then fit the decay time to a first-order exponential curve to calculate tau. From there, we can divide tau by Rin to get the Cm value.

In [ ]:
#creating current clamp
iclamp = h.IClamp(GC.soma[0](0.5))

#setting parameters for the current clamp
iclamp.amp = 5 #nA
iclamp.dur = 10 #ms
iclamp.delay = 5 #ms